In [1]:
import os, argparse
from build_nnfps import main as build_nnfps_main
import numpy as np
import pandas as pds
import pickle, math
from preprocessing import data_preparation, _char_set, get_property, canonocalize
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import SSVAE
import torch

/root/miniconda3/envs/torchkick/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Molecule fingerprint regeneration for transfer learning, if necessary

In [2]:
# training datafiles to use, pick the combination as you like
csv_files = ['./data/table_2_MP_IE_EA.csv',
             './data/table_2_ZINC_310k.csv'] 
# 'data/paper_clean_DC.csv' is skipped for now because it has many wrong & unphysical labels because of NLP

out_pkl = 'data/smiles2nn.pkl'
if os.path.isfile(out_pkl):
    os.system('rm '+out_pkl)

first = True
for csv_file in csv_files:
    args = argparse.Namespace()
    args.csv_file = csv_file
    if first:
        args.input_vocab_file = ''
        args.output_vocab_file = out_pkl
        args.fp_check = False
        # build_nnfps_main(args)
        first = False
    else:
        args.input_vocab_file = out_pkl
        args.output_vocab_file = out_pkl
        args.fp_check = False
        # build_nnfps_main(args)

# Prepare Combination of Dirty Data for Training SSVAE Model

In [3]:
# are we doing pre-training version? if yes, change pretrain to True
pretrain = False
pretrain_pkl = 'data/smiles2nn.pkl'

# we will try train generative model with these training data files
if 'csv_files' in locals():
    data_uri = csv_files
else:
    data_uri = ['./data/table_2_MP_IE_EA.csv',
                './data/table_2_ZINC_310k.csv'] 
# 'data/paper_clean_DC.csv' is skipped 

# first check how many data points we have in the training data
tmp_smiles = {}
for csv_in in data_uri:
    data = pds.read_csv( csv_in )
    for i in range(len(data['SMILES'])):
        tmp_smiles[ canonocalize(data['SMILES'][i]) ] = True

# define training, validation, and test set division
ntotal = len(tmp_smiles)
ntrn = math.floor( 0.9 * ntotal )
ntst = ntotal - ntrn
frac_val = 0.05
del data, tmp_smiles

# data preparation
if pretrain:
    data, scaler_Y = data_preparation(data_uri, ntrn, ntst,
                                      frac_val = frac_val,
                                      pretrain_uri = pretrain_pkl)
else:
    data, scaler_Y = data_preparation(data_uri, ntrn, ntst,
                                      frac_val = frac_val)

# tag for dumping intermediate models
dt = '221214034018'
model_dir = 'models/'+dt
tmp_model_tag = model_dir+'/'    
scaler_Y_pkl = 'preprocessed_scaler_Y.pkl'
scaler_Y = pickle.load(open(tmp_model_tag+scaler_Y_pkl,'rb'))

print('::: Data preparation completed')

::: Data preparation completed


# Load Previously Trained Model, if Needed

In [4]:
# pre-defined parameters
beta=10000.
char_set = _char_set()
dim_z = 100
dim_h = 250
n_hidden = 3
batch_size = 100

# tag for dumping intermediate models
save_uri = tmp_model_tag+'model_final.pth.tar'

# Instantiate the model
model = SSVAE.TorchModel(sample_data = data, dim_z = dim_z, dim_h = dim_h,
                         n_hidden = n_hidden, batch_size = batch_size, beta = float(beta), char_set = char_set,
                         tmp_model_tag = tmp_model_tag)
dev = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model.initialize(torch.Tensor(data['trnY']), torch.Tensor(data['trnMask']), dev)
model.load_state_dict( torch.load(save_uri)['state_dict'] )
model.eval()

TorchModel(
  (predictor): bi_GRU_zero(
    (GRU): GRU(43, 250, num_layers=3, batch_first=True, bidirectional=True)
    (linear_out_y): Linear(in_features=500, out_features=10, bias=True)
  )
  (encoder): bi_GRU_init(
    (GRU): GRU(86, 250, num_layers=3, batch_first=True, bidirectional=True)
    (linear_in): Linear(in_features=5, out_features=500, bias=True)
    (linear_peek): Linear(in_features=5, out_features=43, bias=True)
    (linear_out_z): Linear(in_features=500, out_features=200, bias=True)
  )
  (decoder): uni_GRU_init(
    (GRU): GRU(86, 250, num_layers=3, batch_first=True)
    (linear_in): Linear(in_features=105, out_features=250, bias=True)
    (linear_peek): Linear(in_features=105, out_features=43, bias=True)
    (linear_out_x): Linear(in_features=250, out_features=43, bias=True)
  )
  (softmax): Softmax(dim=-1)
  (sigmoid): Sigmoid()
)

# Property Prediction Performance

In [5]:
scaled_tstY_hat = model.predict_routine(sample_data = data).detach().cpu().numpy()
tstY_hat = [scaler_Y[i].inverse_transform(scaled_tstY_hat[:,i:i+1]) for i in range(scaled_tstY_hat.shape[1])]
tstY_hat = np.concatenate(tstY_hat, axis=1)

dim_y = data['dim_y']
tstY = data['tstY']
tstMask = data['tstMask']
Y_names = data['Y_names']
for j in range(dim_y):
    idx = np.where( tstMask[:,j] == 1 )[0]
    print('Label Name:', Y_names[j])
    print([j, mean_absolute_error(tstY[idx,j], tstY_hat[idx,j])])

Label Name: EA
[0, 0.1169005931059209]
Label Name: IE
[1, 0.10702052969920628]
Label Name: LogP
[2, 0.047518822715899446]
Label Name: MolWt
[3, 2.3350616049841486]
Label Name: QED
[4, 0.006287986592629243]


# Unconditional Generation

In [6]:
print('Unconditional')
for t in range(10):
    smi = model.sampling_unconditional()
    print([t, smi, get_property(smi)])

Unconditional
[0, 'C1=Cc2cccc(F)c2O1', [136.032443, 2.5719000000000003, 0.5420726230197014]]
[1, 'CNc1ccc(C(=O)Nc2ccncc2)s1', [233.062282972, 2.4371, 0.8557226676000261]]
[2, 'CCC(C)N(Cc1cccc(OC)c1)C(=O)CN(c1cc(C)ccc1C)S(C)(=O)=O', [432.20827850399996, 3.9053400000000034, 0.6026273328571766]]
[3, 'CNc1nnc(-c2ccccc2)o1', [175.074561908, 1.7782999999999998, 0.7553649218675917]]
[4, 'C#CCNC(=O)c1cc([N+](=O)[O-])cs1', [210.009913052, 1.0193, 0.4595838004113913]]
[5, 'CN1CCN(C(=O)c2cn(-c3ccccc3)c3ccccc23)c2ccccc21', [367.168462292, 4.727100000000004, 0.5125046791114485]]
[6, 'CC(C)(C)Nc1ccc([N+](=O)[O-])cn1', [195.100776656, 2.2001999999999997, 0.5798279247654079]]
[7, 'CCCC(=O)NC1CCCc2ccccc21', [217.146664228, 2.9803000000000015, 0.8282139229114488]]
[8, 'CC(C)c1noc(CCC(=O)Nc2ccc(Br)cc2)n1', [337.04258885200005, 3.5268000000000024, 0.9056004868084414]]
[9, 'NC(=O)C1(O)CCCC1', [129.078978592, -0.22320000000000073, 0.5136390373603947]]


# Conditional Generation

In [7]:
def print_log(text):
    outfile = 'gen_log.txt'
    if os.path.isfile(outfile):
        open(outfile,'a').write(text+'\n')
    else:
        open(outfile,'w').write(text+'\n')
    return

print('Conditional')
# Determine the property values we want to use for conditional generation
ynames = ['MolWt']
yids = [Y_names.index(yname) for yname in ynames]

ytargets = [250.0]
ymeans = np.array([scaler_Y[yid].mean_[0] for yid in yids])
ystds = np.array([np.sqrt(scaler_Y[yid].var_[0]) for yid in yids])
ytargets_transform = ( np.array(ytargets) - ymeans ) / ystds
                        
print(ynames, ':', ytargets)
for t in range(30000):
    smi = model.sampling_conditional(yids, ytargets_transform)
    props = get_property(smi)
    print([t, smi, props])
    print_log(str([t, smi, props]))
print('')

Conditional
['MolWt'] : [250.0]
[0, 'COc1cccc(-c2ccc(C(C)(C)C)cc2)c1C', [254.167065324, 4.968120000000004, 0.7350158415545001]]
[1, 'CCc1nc(SCC(=O)c2ccccc2)n[nH]1', [247.077933036, 2.3420999999999994, 0.650523977357497]]
[2, 'Cc1cccc(N2C(=O)C3C4C=CC(O4)C3C2=O)n1', [256.084792244, 0.8328199999999999, 0.5514221635047588]]
[3, 'CC(C)CNC(=O)Nc1ccc(-n2ccnc2)cc1', [258.148061196, 2.649800000000001, 0.8853392335109673]]
[4, 'CC1(C)CC(=O)C(=Cc2cccc([N+](=O)[O-])c2)C1=O', [259.084457896, 2.5463000000000005, 0.3536905243368692]]
[5, 'COc1ccccc1Cn1nc(C)c(C(C)C)n1', [245.152812228, 2.766820000000001, 0.8311843169297775]]
[6, 'Cc1ccc(C(=O)N2CCCC(C(=O)O)C2)s1', [253.07726434, 1.99332, 0.8765801166482484]]
[7, 'CC(C)CCNC(=O)CN1C(=O)c2ccccc21', [246.136827816, 1.8091, 0.8610201348546711]]
[8, 'COc1ccccc1CC(=O)NC1CCCCC1', [247.157228912, 2.6866000000000003, 0.8880220109473738]]
[9, 'CS(=O)(=O)C1CN(Cc2ccccc2)CC1C', [253.113649848, 1.5514999999999999, 0.819997528326135]]
[10, 'COc1cc(C)ccc1OCC(=O)NC(C)(C

[04:07:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 13 14


[41, 'CS(=O)(=O)CCNC(=O)c1ccccc1F', [245.052192464, 0.6000999999999994, 0.8445361001230003]]
[42, 'Cc1cc(C)n(CCNC(=O)c2nccn2C)n1', [247.14331016399998, 0.6634399999999998, 0.8630004530846473]]
[43, 'Cc1nccn1-c1ccc(CNC(=O)C2CC2)cc1', [255.137162164, 2.20692, 0.9101143922221602]]
[44, 'Cc1cc2ccccc2n(CC(=O)N(C)C)c1=O', [244.121177752, 1.3981199999999998, 0.8009953695920096]]
[45, 'COc1ccc(C(=O)NCc2cccnc2)cc1C', [256.121177752, 2.32862, 0.9130212461250495]]
[46, 'Cc1cc(OCCOCCOC)c2c(C)noc12', 'invalid']


[04:07:48] Can't kekulize mol.  Unkekulized atoms: 1 2 3 12 13 15 17


[47, 'CCCS(=O)(=O)N1CCCCC1CC(=O)O', [249.103479088, 1.0554000000000001, 0.7901081249894709]]
[48, 'COc1ccc(N2C(=O)c3ccccc3C2=O)cc1', [253.073893212, 2.495800000000001, 0.7723859123595891]]
[49, 'CN(C)c1ccc(C(=O)Nc2ccc(F)cc2)o1', [248.096105876, 2.737000000000001, 0.9079562893457663]]
[50, 'Cc1[nH]c2ccccc2c1CC(=O)N1CCOC1=O', [258.100442308, 1.9976199999999997, 0.8956922224415933]]
[51, 'O=C(CCC1CCOCC1)N1CCc2cccn21', [248.15247788, 1.7156, 0.8159302619077964]]
[52, 'CCOC(C)C(=O)c1ccc(Cl)c(Cl)c1', [246.021434984, 3.6011000000000024, 0.7595637811032594]]
[53, 'COCC(=O)Nc1ccc(OC)c(C(=O)OC)c1', [253.09502258, 1.0667, 0.7943275387329601]]
[54, 'CCC(C)CC(=O)Nc1ccc2c(c1)CCCC2', [245.177964356, 3.940100000000003, 0.8549412468412371]]
[55, 'O=C(Nc1ccc([N+](=O)[O-])cc1)c1cccs1', [248.025563116, 2.9086000000000007, 0.6700840464841774]]
[56, 'Cc1ccc(C(=O)CCC(=O)c2ccccn2)nc1', [254.105527688, 2.630820000000001, 0.7694463937414283]]
[57, 'CCOC(=O)C1CC(=O)N(CC2CCCC2)C1=O', [253.131408088, 1.1148, 0.427

...

[29873, 'O=C(NCCCn1ccnn1)c1cccc(F)c1', [248.107339256, 1.2372999999999996, 0.8096898634356203]]
[29874, 'C=C(C)CNC(=S)NCCOc1ccccc1', [250.113984196, 2.1056, 0.46010934963537925]]
[29875, 'CC(C)(C)OC(=O)C=Cc1ccc(Cl)c(C)c1', [252.091707464, 4.003320000000003, 0.5873942888337134]]
[29876, 'CC(C)c1cc(C(=O)N2CCCC(O)C2)ccn1', [248.15247788, 1.8019, 0.8679781017313054]]
[29877, 'Nc1c(F)c(F)c(F)c(F)c1Br', [242.930674044, 2.5877, 0.3225042554692912]]
[29878, 'CC(C)C(NC(=O)OCc1ccccc1)C(N)=O', [250.131742436, 1.4226999999999999, 0.8288584562022728]]
[29879, 'Cc1n[nH]c(C)c1C(=O)C=Cc1ccccc1F', [244.101191256, 3.0617400000000012, 0.6658584198883459]]
[29880, 'Cc1nn(C)c(C)c1C(=O)Nc1ccc(F)cc1', [247.112090288, 2.4283400000000004, 0.8855167228409315]]
[29881, 'CC(C(=O)NCCC(=O)O)c1ccc(F)cc1C', [253.111421592, 1.8285200000000001, 0.8408847051856132]]
[29882, 'Fc1ccc2ncc(-c3ccc(Cl)cc3)n2c1', [246.036004156, 3.793800000000002, 0.6385188876303342]]
[29883, 'O=C1CC(c2nc(-c3cccs3)no2)CCN1', [249.057197592, 1.

[07:22:47] Explicit valence for atom # 4 B, 4, is greater than permitted


[29936, 'COC(=O)c1ccc(OCCOCCOC)cc1', [254.115423676, 1.5150000000000001, 0.5197308448689055]]
[29937, 'CCOc1ccc(-c2nnc(SC)n2C)cc1', [249.0935831, 2.6027000000000005, 0.7808660257164696]]
[29938, 'COc1ccc(C=CC(=O)c2ccc(C)cc2)cc1', [252.115029752, 3.899720000000003, 0.6090499614418475]]
[29939, 'Fc1cccc(CN2CCCC2c2ccccc2)c1', [255.142327796, 4.162800000000003, 0.7955704916344388]]
[29940, 'O=C(NC1CCCCC1O)c1ccc(F)cc1F', [255.10708516, 1.9981, 0.8487929694011956]]
[29941, 'COc1cc(C=O)ccc1OCC(=O)NC(C)=O', [251.079372516, 0.5491999999999999, 0.772649119650181]]
[29942, 'CC1(c2cccc([N+](=O)[O-])c2)CCSCCO1', [253.07726434, 2.9635000000000016, 0.6003367590995757]]
[29943, 'O=C(NCCc1ccccc1)C(=O)N1CCCO1', [248.116092372, 0.5091999999999997, 0.7928396339204605]]
[29944, 'Cc1ccc(N2CCN(CC(=O)O)CC2=O)cc1', [248.116092372, 0.7282199999999999, 0.8562463585821275]]
[29945, 'Cc1ccc(NC(=O)c2cc(C)nn2C)cc1F', [247.112090288, 2.4283400000000004, 0.8855167228409315]]
[29946, 'Cc1cccc(COC(=O)c2cnn(C)c2C)c1', [2

[07:25:07] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 5 15 16


[29979, 'CC(=O)C(C(=O)OC)C(=O)c1ccc(Cl)cc1', [254.034586512, 1.9008999999999998, 0.4677600242737181]]
[29980, 'Cc1ccnc(N2CCCC(C(=O)N(C)C)C2)c1', [247.168462292, 1.6946200000000002, 0.7977683892569003]]
[29981, 'NCC1CCN(C(=O)Cc2ccc(F)cc2)CC1', [250.148141448, 1.5655, 0.8847010925499526]]
[29982, 'COc1cc(C(=O)CC(C)(C)C)cc(OC)c1O', [252.13615912, 3.028300000000002, 0.8367494421378483]]
[29983, 'Cc1ccc(C)c(N2C(=O)C3CC=CCC3C2=O)c1', [255.125928784, 2.7590400000000006, 0.5713249725499708]]
[29984, 'OCC(CC(F)(F)F)C(F)(F)C(F)(F)F', [246.029090572, 2.7449000000000003, 0.758895182549007]]
[29985, 'Nc1cncc(C2CCN(CC3CCOC3)C2)c1', [247.168462292, 1.4896, 0.8789416184104104]]
[29986, 'CCN(CC)C(=O)c1cc([N+](=O)[O-])ccc1Cl', [256.061469956, 2.730200000000001, 0.6145786337143463]]
[29987, 'CCCCNC(=O)c1cc(S(C)(=O)=O)co1', [245.07217896, 1.2130000000000003, 0.7910363178560784]]
[29988, 'CC(=O)N1CCCCC1c1nc2ccccc2o1', [244.121177752, 2.9013000000000018, 0.7744778797149376]]
[29989, 'CCC1c2cccn2CCN1Cc1nccn1